# 🦛 Chonkie: RecursiveChunker for PDF and Markdown Chunking

![](https://drive.google.com/uc?export=view&id=1FHBnC4CgPjrvSvoXwJiZqg56I8Ck3SbD)

In this notebook, we go over how you can use Chonkie to quickly parse, chunk and run RAG on PDF and Markdown files!

## Overview

Our entire workflow can be summarized in four simple steps (See table of contents sidebar to jump to a step):

1. Convert PDF document to Markdown
2. With Chonkie's RecursiveChunker, chunk our markdown document
3. Prepare Chunks for RAG
4. Use our generated data with an LLM!

## Acknowledgments

We use some awesome services in this notebook, be sure to check them out too:

- Chonkie (of course)
- Arxiv (for our example PDF file)
- Docling (for PDF to Markdown conversion)
- Model2Vec (for embeddings)
- Vicinity (for indexing)
- Together Client (for LLM calls)

## Installs and Imports

Installation might take a while, but only the first time!

**You may have to restart on your colab instance.** Don't worry, that's completely normal!

Just grab a snack and await the CHONK!

In [1]:
!pip install -q chonkie docling model2vec vicinity together rich[jupyter]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
from chonkie import RecursiveChunker, RecursiveRules, RecursiveLevel
from docling.document_converter import DocumentConverter
from google.colab import userdata
from model2vec import StaticModel
import numpy as np
import os
from pprint import pprint
from rich.console import Console
from rich.text import Text
from together import Together
from transformers import AutoTokenizer
from typing import List
from vicinity import Vicinity, Backend, Metric

## Inits and Utils

Lets setup everything we need for a smooth chunking experience

### Define utilty functions

Just some functions to make the `.pynb` experience better :D

In [3]:
# Rich text console for better printing
console = Console()

# A wrapper to pretty print
def rprint(text: str, console: Console=console, width: int = 80) -> None:
  richtext = Text(text)
  console.print(richtext.wrap(console, width=width))

### Initialize Your Model!

Lets initialize the model we will build our application with!

Here, we are going to be using Deepseek's R1 model.

In [ ]:
# Set your Together API key to use Deepseek R1 with it~
os.environ['TOGETHER_API_KEY'] = userdata.get('TOGETHER_API_KEY')

# Initialise a model2vec model for encoding sentences for retrieval
model = StaticModel.from_pretrained("minishlab/potion-retrieval-32M")

# Initialise the Together client to call upon Deepseek R1
client = Together()

# (Optional) Initialise the tokenizer for Deepseek R1
# We use this to get token counts at various points in this colab.
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/16.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.85M [00:00<?, ?B/s]

## Step 1: Use docling to convert from PDF to Markdown!

**Note**: This step can take about 20-30 seconds depending on your PDF.

In [4]:
# Docling can convert any PDF to markdown!
converter = DocumentConverter()
source = "https://arxiv.org/pdf/1706.03762"
result = converter.convert(source)
text = result.document.export_to_markdown()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
# @title A quick look at the text we'll be working with~
rprint(text)

Provided proper attribution is provided, Google hereby grants permission to 
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.

## Attention Is All You Need

Ashish Vaswani ∗ Google Brain avaswani@google.com

Noam Shazeer ∗ Google Brain noam@google.com

Niki Parmar ∗ Google Research nikip@google.com

Jakob Uszkoreit ∗ Google Research usz@google.com

Llion Jones ∗ Google Research llion@google.com

Aidan N. Gomez ∗ † University of Toronto

aidan@cs.toronto.edu

Łukasz Kaiser Google Brain lukaszkaiser@google.com

∗

Illia Polosukhin ∗ ‡

illia.polosukhin@gmail.com

## Abstract

The dominant sequence transduction models are based on complex recurrent or 
convolutional neural networks that include an encoder and a decoder. The best 
performing models also connect the encoder and decoder through an attention 
mechanism. We propose a new simple network architecture, the Transformer, based 
solely on attention mechanisms, dispensing with recurrence and convolutions 
entirely. Experiments on two machine translation tasks show these models to be 
superior in quality while being more parallelizable and requiring significantly 
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 
Englishto-German translation task, improving over the existing best results, 
including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French 
translation task, our model establishes a new single-model state-of-the-art BLEU
score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the
training costs of the best models from the literature. We show that the 
Transformer generalizes well to other tasks by applying it successfully to 
English constituency parsing both with large and limited training data.

∗ Equal contribution. Listing order is random. Jakob proposed replacing RNNs 
with self-attention and started the effort to evaluate this idea. Ashish, with 
Illia, designed and implemented the first Transformer models and has been 
crucially involved in every aspect of this work. Noam proposed scaled 
dot-product attention, multi-head attention and the parameter-free position 
representation and became the other person involved in nearly every detail. Niki
designed, implemented, tuned and evaluated countless model variants in our 
original codebase and tensor2tensor. Llion also experimented with novel model 
variants, was responsible for our initial codebase, and efficient inference and 
visualizations. Lukasz and Aidan spent countless long days designing various 
parts of and implementing tensor2tensor, replacing our earlier codebase, greatly
improving results and massively accelerating our research.

† Work performed while at Google Brain.

‡ Work performed while at Google Research.

## 1 Introduction

Recurrent neural networks, long short-term memory [13] and gated recurrent [7] 
neural networks in particular, have been firmly established as state of the art 
approaches in sequence modeling and transduction problems such as language 
modeling and machine translation [35, 2, 5]. Numerous efforts have since 
continued to push the boundaries of recurrent language models and 
encoder-decoder architectures [38, 24, 15].

Recurrent models typically factor computation along the symbol positions of the 
input and output sequences. Aligning the positions to steps in computation time,
they generate a sequence of hidden states h t , as a function of the previous 
hidden state h t -1 and the input for position t . This inherently sequential 
nature precludes parallelization within training examples, which becomes 
critical at longer sequence lengths, as memory constraints limit batching across
examples. Recent work has achieved significant improvements in computational 
efficiency through factorization tricks [21] and conditional computation [32], 
while also improving model performance in case of the latter. The fundamental 
constraint of sequential computation, however, remains.

Atten

In [6]:
from IPython.display import HTML
HTML(f"<button onclick=navigator.clipboard.writeText('{text}')>Copy</button>")

### Get the total token counts for this PDF

Our example PDF is made up of ~**9,865** tokens! Keep this in mind as we move forward ;)

In [ ]:
total_text_tokens = len(tokenizer.encode(text))
rprint(f"This PDF contains: {total_text_tokens} tokens")

This PDF contains: 9865 tokens

## Step 2: Chunk your texts w/ Chonkie!

### Initalize your Chunker!
For effective markdown chunking, we will be using Chonkie's **recurisve chunker**!

With the recursive chunker, we can define custom RecursiveRules that fit our file's structure and format. Since we are working with Markdown files, we will specify rules that help Chonkie understand it's syntax.

In [ ]:
rules = RecursiveRules(
    levels=[
        RecursiveLevel(delimiters=['######', '#####', '####', '###', '##', '#']),
        RecursiveLevel(delimiters=['\n\n', '\n', '\r\n', '\r']),
        RecursiveLevel(delimiters='.?!;:'),
        RecursiveLevel()
    ]
)
chunker = RecursiveChunker(rules=rules, chunk_size=384)

### Lets Chunk!

In [ ]:
# This is all it takes to chunk!
chunks = chunker(text)
print(f"Total number of chunks: {len(chunks)}")

Total number of chunks: 57


In [ ]:
# @title A quick look at our chunks~
for chunk in chunks[:4]:
  rprint(chunk.text)
  print('-'*80, '\n\n')

Provided proper attribution is provided, Google hereby grants permission to 
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.

## Attention Is All You Need

Ashish Vaswani ∗ Google Brain avaswani@google.com

Noam Shazeer ∗ Google Brain noam@google.com

Niki Parmar ∗ Google Research nikip@google.com

Jakob Uszkoreit ∗ Google Research usz@google.com

Llion Jones ∗ Google Research llion@google.com

Aidan N. Gomez ∗ † University of Toronto

aidan@cs.toronto.edu

Łukasz Kaiser Google Brain lukaszkaiser@google.com

∗

Illia Polosukhin ∗ ‡

illia.polosukhin@gmail.com

##

-------------------------------------------------------------------------------- 




Abstract

The dominant sequence transduction models are based on complex recurrent or 
convolutional neural networks that include an encoder and a decoder. The best 
performing models also connect the encoder and decoder through an attention 
mechanism. We propose a new simple network architecture, the Transformer, based 
solely on attention mechanisms, dispensing with recurrence and convolutions 
entirely. Experiments on two machine translation tasks show these models to be 
superior in quality while being more parallelizable and requiring significantly 
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 
Englishto-German translation task, improving over the existing best results, 
including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French 
translation task, our model establishes a new single-model state-of-the-art BLEU
score of 41.8 after training for 3.5 days on eight GPUs, a small fraction of the
training costs of the best models from the literature. We show that the 
Transformer generalizes well to other tasks by applying it successfully to 
English constituency parsing both with large and limited training data.

##

-------------------------------------------------------------------------------- 




1 Introduction

Recurrent neural networks, long short-term memory [13] and gated recurrent [7] 
neural networks in particular, have been firmly established as state of the art 
approaches in sequence modeling and transduction problems such as language 
modeling and machine translation [35, 2, 5]. Numerous efforts have since 
continued to push the boundaries of recurrent language models and 
encoder-decoder architectures [38, 24, 15].

Recurrent models typically factor computation along the symbol positions of the 
input and output sequences. Aligning the positions to steps in computation time,
they generate a sequence of hidden states h t , as a function of the previous 
hidden state h t -1 and the input for position t . This inherently sequential 
nature precludes parallelization within training examples, which becomes 
critical at longer sequence lengths, as memory constraints limit batching across
examples. Recent work has achieved significant improvements in computational 
efficiency through factorization tricks [21] and conditional computation [32], 
while also improving model performance in case of the latter. The fundamental 
constraint of sequential computation, however, remains.

Attention mechanisms have become an integral part of compelling sequence 
modeling and transduction models in various tasks, allowing modeling of 
dependencies without regard to their distance in the input or output sequences 
[2, 19]. In all but a few cases [27], however, such attention mechanisms are 
used in conjunction with a recurrent network.

In this work we propose the Transformer, a model architecture eschewing 
recurrence and instead relying entirely on an attention mechanism to draw global
dependencies between input and output. The Transformer allows for significantly 
more parallelization and can reach a new state of the art in translation quality
after being trained for as little as twelve hours on eight P100 GPUs.

##

-------------------------------------------------------------------------------- 




2 Background

The goal of reducing sequential computation also forms the foundation of the 
Extended Neural GPU [16], ByteNet [18] and ConvS2S [9], all of which use 
convolutional neural networks as basic building block, computing hidden 
representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or 
output positions grows in the distance between positions, linearly for ConvS2S 
and logarithmically for ByteNet. This makes it more difficult to learn 
dependencies between distant positions [12]. In the Transformer this is reduced 
to a constant number of operations, albeit at the cost of reduced effective 
resolution due to averaging attention-weighted positions, an effect we 
counteract with Multi-Head Attention as described in section 3.2.

Self-attention, sometimes called intra-attention is an attention mechanism 
relating different positions of a single sequence in order to compute a 
representation of the sequence. Self-attention has been used successfully in a 
variety of tasks including reading comprehension, abstractive summarization, 
textual entailment and learning task-independent sentence representations [4, 
27, 28, 22].

End-to-end memory networks are based on a recurrent attention mechanism instead 
of sequencealigned recurrence and have been shown to perform well on 
simple-language question answering and language modeling tasks [34].

To the best of our knowledge, however, the Transformer is the first transduction
model relying entirely on self-attention to compute representations of its input
and output without using sequencealigned RNNs or convolution. In the following 
sections, we will describe the Transformer, motivate self-attention and discuss 
its advantages over models such as [17, 18] and [9].

##

-------------------------------------------------------------------------------- 




## Step 3: Retrieval Augmented Generation!

Let's setup RAG to get our chunks when needed

### Step 3.1: Get the embeddings for each of the chunks!

In [ ]:
items = [chunk.text for chunk in chunks]
vectors = model.encode(items)
print(vectors.shape)

(57, 512)


### Step 3.2: Create an index with the chunks and embeddings for retrieval

In [ ]:
# Initialize the Vicinity instance (using basic backend and cosine metric)
vicinity = Vicinity.from_vectors_and_items(
    vectors=vectors,
    items=items,
    backend_type=Backend.BASIC,
    metric=Metric.COSINE
)

### Step 3.3: Pack it all together in a single retrieval function!

Given a query, retrieve all relevant embeddings~

In [ ]:
def get_embeddings(query: str):
  query_vector = model.encode(query)
  results = vicinity.query(query_vector, k=4)
  return [x[0] for x in results[0]]

### (Optional) Step 3.4: Test our function

In [ ]:
query = "What is a Multi-Head Self Attention?"
retrieved_chunks = get_embeddings(query)

for chunk in retrieved_chunks:
  rprint(chunk)
  print('-'*80, '\n\n')

Attention Visualizations Input-Input Layer5

Figure 3: An example of the attention mechanism following long-distance 
dependencies in the encoder self-attention in layer 5 of 6. Many of the 
attention heads attend to a distant dependency of the verb 'making', completing 
the phrase 'making...more difficult'. Attentions here shown only for the word 
'making'. Different colors represent different heads. Best viewed in color.

<!-- image -->

Input-Input Layer5

Figure 4: Two attention heads, also in layer 5 of 6, apparently involved in 
anaphora resolution. Top: Full attentions for head 5. Bottom: Isolated 
attentions from just the word 'its' for attention heads 5 and 6. Note that the 
attentions are very sharp for this word.

<!-- image -->

Input-Input Layer5

Figure 5: Many of the attention heads exhibit behaviour that seems related to 
the structure of the sentence. We give two such examples above, from two 
different heads from the encoder self-attention at layer 5 of 6. The heads 
clearly learned to perform different tasks.

<!-- image -->

-------------------------------------------------------------------------------- 




2 Background

The goal of reducing sequential computation also forms the foundation of the 
Extended Neural GPU [16], ByteNet [18] and ConvS2S [9], all of which use 
convolutional neural networks as basic building block, computing hidden 
representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or 
output positions grows in the distance between positions, linearly for ConvS2S 
and logarithmically for ByteNet. This makes it more difficult to learn 
dependencies between distant positions [12]. In the Transformer this is reduced 
to a constant number of operations, albeit at the cost of reduced effective 
resolution due to averaging attention-weighted positions, an effect we 
counteract with Multi-Head Attention as described in section 3.2.

Self-attention, sometimes called intra-attention is an attention mechanism 
relating different positions of a single sequence in order to compute a 
representation of the sequence. Self-attention has been used successfully in a 
variety of tasks including reading comprehension, abstractive summarization, 
textual entailment and learning task-independent sentence representations [4, 
27, 28, 22].

End-to-end memory networks are based on a recurrent attention mechanism instead 
of sequencealigned recurrence and have been shown to perform well on 
simple-language question answering and language modeling tasks [34].

To the best of our knowledge, however, the Transformer is the first transduction
model relying entirely on self-attention to compute representations of its input
and output without using sequencealigned RNNs or convolution. In the following 
sections, we will describe the Transformer, motivate self-attention and discuss 
its advantages over models such as [17, 18] and [9].

##

-------------------------------------------------------------------------------- 




Scaled Dot-Product Attention

<!-- image -->

Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention 
consists of several attention layers running in parallel.

<!-- image -->

of the values, where the weight assigned to each value is computed by a 
compatibility function of the query with the corresponding key.

##

-------------------------------------------------------------------------------- 




3.2.2 Multi-Head Attention

Instead of performing a single attention function with d model-dimensional keys,
values and queries, we found it beneficial to linearly project the queries, keys
and values h times with different, learned linear projections to d k , d k and d
v dimensions, respectively. On each of these projected versions of queries, keys
and values we then perform the attention function in parallel, yielding d v 
-dimensional

output values. These are concatenated and once again projected, resulting in the
final values, as depicted in Figure 2.

Multi-head attention allows the model to jointly attend to information from 
different representation subspaces at different positions. With a single 
attention head, averaging inhibits this.

<!-- formula-not-decoded -->

<!-- formula-not-decoded -->

Where the projections are parameter matrices W Q i ∈ R d model × d k , W K i ∈ R
d model × d k , W V i ∈ R d model × d v and W O ∈ R hd v × d model .

In this work we employ h = 8 parallel attention layers, or heads. For each of 
these we use d k = d v = d model /h = 64 . Due to the reduced dimension of each 
head, the total computational cost is similar to that of single-head attention 
with full dimensionality.

##

-------------------------------------------------------------------------------- 




## Step 4: Lets build our LLM Application!

Our data is ready! Let's setup our LLM application to answer user queries

In [ ]:
# A simple function to make LLM prompts with chunks
def create_prompt(chunks: List[str], query: str) -> str:
  prompt_template = """<instructions>
  Based on the provided contexts, answer the given question to the best of your ability. Remember to also add citations at appropriate points in the format of square brackets like [1][2][3], especially at sentence or paragraph endings.
  You will be given 4 passages in the context, marked with a label 'Doc [1]:' to denote the passage number. Use that number for citations. Answer only from the given context, and if there's no appropriate context, reply "No relevant context found!".
  </instructions>

  <context>
  {context}
  </context>

  <query>
  {query}
  </query>
  """
  context = "\n\n".join([f"Doc {i+1}: {chunk}" for i, chunk in enumerate(chunks)])
  prompt = prompt_template.format(context=context, query=query)
  return prompt

In [ ]:
# Prompt to use
query = "What is a Multi-Head Self Attention?"
retrieved_chunks = get_embeddings(query)
prompt = create_prompt(retrieved_chunks, query)

In [ ]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=[{"role": "user", "content": prompt}],
)
# Print the final response without the thinking tokens
answer = response.choices[0].message.content.split("</think>")[-1]
rprint(answer)

Multi-Head Self Attention is a mechanism in the Transformer model that enables 
the system to jointly process information from different representation 
subspaces at various positions in parallel. Instead of using a single attention 
function, it linearly projects the queries, keys, and values multiple times 
(using distinct learned projections for each "head") [4]. Each head 
independently performs the scaled dot-product attention function on these 
projected inputs, generating outputs that are concatenated and reprojected to 
form the final result [4][3]. 

This approach allows the model to capture diverse dependencies, such as 
long-distance syntactic relationships (e.g., verb-phrase completion in Figure 3 
of Doc 1) or anaphora resolution (e.g., tracking "its" in Figure 4 of Doc 1) 
[1]. By using multiple heads (typically 8), the model avoids the limitations of 
single-head attention, where averaging might suppress nuanced patterns across 
different positional or contextual subspaces [4]. The computational cost remains
manageable due to reduced dimensionality per head (e.g., 64 dimensions per head 
when using 8 heads) [4].

### Wow, the model works great! And how many tokens did we use?

In [ ]:
prompt_tokens = len(tokenizer.encode(prompt))
rprint(f"This prompt contains: {prompt_tokens} tokens")

This prompt contains: 1069 tokens

**1,069 tokens!** Rememeber from earlier--without chunking we'd have used 9,865 tokens!

## Conclusions

We got a great answer back from our model, while saving about **~8,800 input tokens**! Chonkie is one efficient hippo 🚀

Hope you found this notebook useful! If you want to learn more about Chonkie, check our [GitHub](https://github.com/chonkie-ai/chonkie), [Twitter](https://x.com/ChonkieAI), and [Bluesky](https://bsky.app/profile/chonkieai.bsky.social)!

If you have any questions about anything we showed us here, reach us at `support@chonkie.ai` or on our [Discord](https://discord.gg/6V5pqvqsCY)

# Fin

Happy Chonking! 🦛✨